In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import mailbox

In [14]:

mailboxfile = "C:/Users/HP/takeout-20220527T154953Z-001/Takeout/Mail/All mail Including Spam and Trash.mbox"

mbox = mailbox.mbox(mailboxfile)
mbox

In [15]:
for key in mbox[0].keys():
    print(key)

X-GM-THRID
X-Gmail-Labels
Delivered-To
Received
X-Received
ARC-Seal
ARC-Message-Signature
ARC-Authentication-Results
Return-Path
Received
Received-SPF
Authentication-Results
DKIM-Signature
X-Google-DKIM-Signature
X-Gm-Message-State
X-Google-Smtp-Source
MIME-Version
X-Received
Date
List-Unsubscribe
X-Notifications
X-Notifications-Bounce-Info
Message-ID
Subject
From
To
Content-Type


In [16]:
import csv

In [17]:
with open('mailbox.csv', 'w') as outputfile:
    writer = csv.writer(outputfile)
    writer.writerow(['subject','from','date','to','label','thread'])
    for message in mbox:
        writer.writerow([
            message['subject'],
            message['from'],
            message['date'],
            message['to'],
            message['X-Gmail-Labels'],
            message['X-GM-THRID']
        ])
        

In [18]:
dfs = pd.read_csv('mailbox.csv', names=['subject', 'from', 'date', 'to',
'label', 'thread'])

In [19]:
dfs

,subject,from,date,to,label,thread
0,subject,from,date,to,label,thread
1,=?UTF-8?Q?3_new_jobs_for_=E2=80=98data_scienti...,Job alerts from Google <notify-noreply@google....,"Tue, 24 May 2022 12:27:26 -0700",ayosamobitade@gmail.com,"Inbox,Opened,Category Updates",1733736999130792636
2,Your program recommendations for the week,"""Will at edX"" <edX@news.edx.org>","Thu, 26 May 2022 14:00:10 +0000",ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733897612589550289
3,=?utf-8?Q?Excel=20Newsletter=20=2D=20STOP=20Em...,=?utf-8?Q?My=20Online=20Training=20Hub?= <webs...,"Thu, 26 May 2022 12:06:07 +0000",<Ayosamobitade@gmail.com>,"Inbox,Category Updates,Unread",1733890522517290196
4,=?UTF-8?Q?*Konga_Food_is_now_live_in_Lagos!*_=...,Konga Food <Newsletter@info.konga.com>,"Mon, 23 May 2022 15:25:50 +0530",ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1733610439749409235
...,...,...,...,...,...,...
21562,Declined Card Transaction - Insufficient Funds,"""GeNS@gtbank.com"" <noreply@gtbank.com>","Sun, 18 Jul 2021 08:16:30 +0100",<ayosamobitade@gmail.com>,"Inbox,Category Updates,Unread",1705605969799682813
21563,Ready to Make Your Career Goals a Reality?,"""Kaplan"" <kaplan@mail.kaptest.com>",16 Dec 2021 07:27:57 -0800,ayosamobitade@gmail.com,"Inbox,Category Promotions,Unread",1719317334410978885
21564,=?utf-8?q?Are_PhDs_a_scam=3F?=,Quora Digest <english-digest-noreply@quora.com>,"Wed, 18 Aug 2021 17:29:08 +0000",ayosamobitade@gmail.com,"Inbox,Category Updates,Unread",1708453002325720055
21565,=?UTF-8?Q?Instablog9ja_Tweeted:_Shock_as_swar?...,Twitter <info@twitter.com>,"Fri, 01 Apr 2022 14:38:18 +0000",Samworld Press <ayosamobitade@gmail.com>,"Inbox,Category Social,Unread",1728917168752283375
